![](../images/TQ42_Banner.png)


# Welcome to TQ42!

## Example how to use channels with CVA_OPT


In [ ]:
from tq42.client import TQ42Client
from tq42.experiment_run import ExperimentRun
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRunStatusProto
from tq42.algorithm import AlgorithmProto
from tq42.compute import HardwareProto
import json
import pandas
import asyncio
import numpy as np
from tq42.channel import Channel, Ask, Tell
import OptimizationTestFunctions as otf

### Get your experiment id by descending the organization, project, experiment hierarchy

In [ ]:
tq42client = TQ42Client()
tq42client.login()
org_list = list_all_organizations(tq42client)
project_list = list_all_projects(client=tq42client, organization_id=org_list[0].id)
experiment_list = list_all_experiments(client=tq42client, project_id=project_list[0].id)
experiment_id = experiment_list[0].id

In [ ]:
print(f"Running experiment within: Org {org_list[0].id}, Proj {project_list[0].id} and Exp {experiment_id}`")

### Single-objective example Rosenbrock function

#### Define your optimization run by setting
* objectives
* variables
* parameters for CVA_OPT
* set up channel and callback function

In [ ]:
cva_params = {}

##### Objectives
* the name of our objective is called like the function itself: Sphere
* the aim_type sets if the objective should be minimized (MINIMIZE) of maximized (MAXIMIZE)

In [ ]:
cva_params['objectives'] = [{'name': 'Rosenbrock', 'aim_type':'MINIMIZE'}]

##### Variables
Variables have a name and depending of their type lower and upper bounds or possible class values
in this example we want to optimize a two dimensional sphere function in the box [-1,1]x[-1,1]

In [ ]:
cva_params['variables'] = []
cva_params['variables'].append({'name': 'x1', 'info_real':{'lower_bound':-1.0, 'upper_bound':1.0}})
cva_params['variables'].append({'name': 'x2', 'info_real':{'lower_bound':-1.0, 'upper_bound':1.0}})

##### Parameters for CVA_OPT

In [ ]:
cva_params['parameters'] = {}
# set the number of generations (iterations)
cva_params['parameters']['max_generation'] = 50
# set the ES specific parameters mue and lambda
cva_params['parameters']['mue'] = 2
cva_params['parameters']['lambda'] = 10

Please note that CVA_OPT will use max_generation * lambda many function evaluations, i.e.,
50 * 10 = 500 function evaluations to minimize the Rosenbrock function.

### Set up channel and callback function to evaluate the objective function
CVA_OPT is a black-box optimizer using evolutionary strategies (ES) meaning it does not know anything 
about the internal structure of the objective function which should be optimized. CVA_OPT sends 
candidate solutions to a so called channel implementing an ask and tell interface with a callback function. The channel sends the candidate solutions. There are evaluated by the callback function, i.e., assigning for each candidate the objective value. Then CVA_OPT reads the result from the channel and the next iteration is started.

#### Define a function to run an experiment with channel


In [ ]:
async def run_exp_with_channel(client, experiment_id, cva_params):
    # set up channel
    channel = await Channel.create(client=client)
    # extend cva_params with func_eval_worker_channel_id
    cva_params['func_eval_worker_channel_id'] = channel.id
    
    # create the experiment run
    run = ExperimentRun.create(
        client=client, 
        algorithm=AlgorithmProto.CVA_OPT, 
        experiment_id=experiment_id,
        compute=HardwareProto.SMALL,
        parameters={'parameters': cva_params, 'inputs': {} }
    )
    print(f" starting run with id {run.id}")

    # define the callback function
    async def callback(ask: Ask) -> Tell:
        dim = len(ask.headers)
        func = otf.Rosenbrock(dim)
        y = []
        for parameter in ask.parameters:
            y.append(float(func(np.array(parameter.values))))
        # add result to data

        return Tell(
            parameters=ask.parameters,
            headers=ask.headers,
            results=y
        )
    # define a function to be called after the optimization is finished
    def success():
        pass

    # let the channel wait for connections
    is_finished = False
    max_retries = 10
    retries = 0
    while (not is_finished) and (retries < max_retries):
        await channel.connect(
                callback=callback,
                finish_callback=success,
                max_duration_in_sec=None,
                message_timeout_in_sec=120
            )
        retries += 1
        run_state = run.check().data.status
        if ExperimentRunStatusProto.Name(run_state) in ['CANCELLED', 'COMPLETED', 'FAILED', 'CANCEL_PENDING']:
            is_finished = True
        else:
            print('run is in state ' + ExperimentRunStatusProto.Name(run_state) + ', continuing')
    # return the run to retrieve the result    
    return run

In [ ]:
run = await run_exp_with_channel(tq42client, experiment_id, cva_params)

##### Get the result

In [ ]:
run_result = run.poll()
print('status: ' + ExperimentRunStatusProto.Name(run_result.data.status))

##### Print the result as pandas data frame

In [ ]:
result_df = pandas.DataFrame(json.loads(json.loads(getattr(getattr(run_result.data, 'result'), 'result_json'))['result'])['result'])
result_df